# New York City traffic accidents example 

In [0]:
import sys
from pathlib import Path

# Add 'src' directory to the Python path
sys.path.append(str(Path().resolve().parent / 'src'))

# Import necessary modules
from stkde import stkde
import geopandas as gpd

# Check if optional dependencies are installed
try:
    import matplotlib.pyplot as plt
    import pyarrow.parquet as pq
except ImportError:
    print('Optional dependencies are not installed. Please install them to run this example.')

In [0]:
# Read dataset
nyc_traffic_gdf = gpd.read_parquet('../data/nyc_traffic.parquet')
print(f"Number of accidents in NYC traffic dataset: {len(nyc_traffic_gdf)}")

Number of accidents in NYC traffic dataset: 1783094


Full STKDE run on NYC traffic dataset:

In [0]:
# Calculate Spatio-Temporal KDE for NYC traffic accidents. For 100^3 voxels and bandwidths of 3280 U.S. survey feet (approx. 1 km) and 30 days.
nyc_stkde = stkde(gdf=nyc_traffic_gdf, time_col='days_after_epoch', crs='EPSG:2263', number_of_voxels=(100,100,100), bandwidths=(3280, 3280, 30),
      output_file='./output/nyc_traffic_accidents')

(1/5) Completed preprocessing in 0.71 seconds.
(2/5) Completed computing bandwidths in 0.02 seconds. Attempted no bandwidth combinations
(3/5) Completed grid computation in 0.00 seconds.
(4/5) Completed STKDE calculations in 0.34 seconds.
(5/5) STKDE result has been saved to c:\wSTKDE\notebooks\output\nyc_traffic_accidents.vti
Total compute time: 1.27 seconds.
Total number of voxels calculated: 1000000.
Voxel size (x:1317.55, y:1179.59, t:45.66), using bandwidths: x=3280.0, y=3280.0, t=30.0.


Weighted STKDE run on NYC traffic dataset. Points with no persons injured excluded.

In [0]:
# Create a new GeoDataFrame dropping rows where number of persons injured is zero.
nyc_traffic_gdf_without_zero: gpd.GeoDataFrame = nyc_traffic_gdf[nyc_traffic_gdf['NUMBER OF PERSONS INJURED'] > 0] # type: ignore
print(f"Removed {len(nyc_traffic_gdf) - len(nyc_traffic_gdf_without_zero)}/{len(nyc_traffic_gdf)} rows with zero injuries. Leaving: {len(nyc_traffic_gdf_without_zero)} points.")

# Calculate the same as before, but using number of injuries as weights
weighted_nyc_stkde = stkde(gdf=nyc_traffic_gdf_without_zero, time_col='days_after_epoch', crs='EPSG:2263', number_of_voxels=(100,100,100), bandwidths=(3280, 3280, 30),
      weight_col='NUMBER OF PERSONS INJURED', output_file='./output/nyc_traffic_accidents_weighted')

Removed 1365356/1783094 rows with zero injuries. Leaving: 417738 points.
(1/5) Completed preprocessing in 0.29 seconds.
(2/5) Completed computing bandwidths in 0.00 seconds. Attempted no bandwidth combinations
(3/5) Completed grid computation in 0.00 seconds.
(4/5) Completed STKDE calculations in 0.08 seconds.
(5/5) STKDE result has been saved to c:\wSTKDE\notebooks\output\nyc_traffic_accidents_weighted.vti
Total compute time: 0.57 seconds.
Total number of voxels calculated: 1000000.
Voxel size (x:1304.76, y:1178.97, t:45.66), using bandwidths: x=3280.0, y=3280.0, t=30.0.


## Code used to preprocess the raw file
Below is the code used to preprocess the raw data from the NYC open data portal. Data retrieved from here [link](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/about_data). Due to the large size of the raw dataset, it is not included with the code. Instead, only the compressed and quicker-to-read parquet format is provided. Code below is shown for full reproducibility.

In [0]:
import pandas as pd
import geopandas as gpd

In [0]:
# Read data in
nyc_traffic = pd.read_csv(
    '../data/Motor_Vehicle_Collisions_-_Crashes_20241006.csv', # Change to different path or file name if newer data is available.
    parse_dates=['CRASH DATE'],
    usecols=['CRASH DATE', 'LATITUDE', 'LONGITUDE', 'NUMBER OF PERSONS INJURED']
)

# Drop rows where latitude or longitude are missing
len_total = len(nyc_traffic)
nyc_traffic = nyc_traffic.dropna(subset=['LATITUDE', 'LONGITUDE'])
len_no_na = len(nyc_traffic)
print(f"Dropped {len_total - len_no_na}/{len_total} rows with missing values")

# Transform CRASH DATE, to new column days_after_epoch, where epoch is 01/01/2012
nyc_traffic['days_after_epoch'] = (nyc_traffic['CRASH DATE'] - pd.Timestamp('2012-01-01')).dt.days
print(f"Final date in dataset: {nyc_traffic['CRASH DATE'].max()}")
# Convert to GeoDataFrame
nyc_traffic_gdf = gpd.GeoDataFrame(
    nyc_traffic,
    geometry=gpd.points_from_xy(nyc_traffic.LONGITUDE, nyc_traffic.LATITUDE),
    crs='EPSG:4326'
)
# Convert to EPSG:2263
nyc_traffic_gdf = nyc_traffic_gdf.to_crs('EPSG:2263')

# Remove all points outside of NYC bounds (roughly)
len_before_clip = len(nyc_traffic_gdf)
nyc_traffic_gdf = nyc_traffic_gdf.cx[903691.238365:1057957.154554, 152625.209653:268237.237965]
len_after_clip = len(nyc_traffic_gdf)
print(f"Clipped {len_before_clip - len_after_clip}/{len_before_clip} points outside of NYC bounds")

# Output to file
nyc_traffic_gdf.to_parquet('../data/nyc_traffic.parquet')

Dropped 251097/2124088 rows with missing values
Final date in dataset: 2024-10-01 00:00:00
